<a href="https://colab.research.google.com/github/arminali23/comp_vision_practices/blob/main/SohmaLLM1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install -U transformers accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 50.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 11.9 MB/s eta 0:00:00


In [2]:
import torch
print("MPS available:", torch.backends.mps.is_available())
print("MPS built:", torch.backends.mps.is_built())
print("CUDA available:", torch.cuda.is_available())

MPS available: False
MPS built: False
CUDA available: True


In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

MODEL_ID = "mistralai/Mistral-7B-Instruct-v0.2"

bnb = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, use_fast=True)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    device_map="auto",
    quantization_config=bnb
)

model.eval()
print("Loaded:", MODEL_ID)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Loaded: mistralai/Mistral-7B-Instruct-v0.2


In [5]:
import json, time

INPUT_JSON = {
  "session_id": "test-session-001",
  "timestamp": "2026-01-14T12:00:00Z",
  "modality_bundle": {
    "behavioral": {
      "interaction_intensity": 0.92,
      "error_rate": 0.68,
      "pause_frequency": 0.08,
      "performance_quality": 0.22,
      "temporal_features": {"interaction_instability": 0.55}
    },
    "data_quality": {"confidence": 0.93}
  }
}

prompt = f"""
You are the Mother LLM for an in-game mental state detection system.

Task:
Infer the player's mental states from structured, normalized gameplay signals (0..1).

Hard rules:
- Output VALID JSON only. No markdown. No explanations.
- Allowed labels: ["stress","frustration","engagement","boredom","cognitive_load","calm"].
- Each emotion must contain: label, intensity, confidence.
- intensity and confidence must be numbers in [0,1].
- Multi-label is allowed.

Return JSON in this exact format:
{{"emotions":[{{"label":"...", "intensity":0.0, "confidence":0.0}}]}}

Input:
{json.dumps(INPUT_JSON)}
""".strip()

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

t0 = time.time()
with torch.no_grad():
    out = model.generate(
        **inputs,
        max_new_tokens=200,
        do_sample=False
    )
lat_ms = (time.time() - t0) * 1000

text = tokenizer.decode(out[0], skip_special_tokens=True)

print("Latency(ms):", round(lat_ms, 1))
print(text)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Latency(ms): 19221.2
You are the Mother LLM for an in-game mental state detection system.

Task:
Infer the player's mental states from structured, normalized gameplay signals (0..1).

Hard rules:
- Output VALID JSON only. No markdown. No explanations.
- Allowed labels: ["stress","frustration","engagement","boredom","cognitive_load","calm"].
- Each emotion must contain: label, intensity, confidence.
- intensity and confidence must be numbers in [0,1].
- Multi-label is allowed.

Return JSON in this exact format:
{"emotions":[{"label":"...", "intensity":0.0, "confidence":0.0}]}

Input:
{"session_id": "test-session-001", "timestamp": "2026-01-14T12:00:00Z", "modality_bundle": {"behavioral": {"interaction_intensity": 0.92, "error_rate": 0.68, "pause_frequency": 0.08, "performance_quality": 0.22, "temporal_features": {"interaction_instability": 0.55}}, "data_quality": {"confidence": 0.93}}}

Example output:
{"emotions":[{"label":"engagement", "intensity":0.8, "confidence":0.95}, {"label":"co

In [7]:
import json, time

INPUT_JSON = {
  "session_id": "test-session-001",
  "timestamp": "2026-01-14T12:00:00Z",
  "modality_bundle": {
    "behavioral": {
      "interaction_intensity": 0.92,
      "error_rate": 0.68,
      "pause_frequency": 0.08,
      "performance_quality": 0.22,
      "temporal_features": {"interaction_instability": 0.55}
    },
    "data_quality": {"confidence": 0.93}
  }
}

prompt = f"""
You are the Mother LLM for an in-game mental state detection system.

Task:
Infer the player's mental states from structured, normalized gameplay signals (0..1).

Hard rules:
- Output VALID JSON only. No markdown. No explanations.
- Allowed labels: ["stress","frustration","engagement","boredom","cognitive_load","calm"].
- Each emotion must contain: label, intensity, confidence.
- intensity and confidence must be numbers in [0,1].
- Multi-label is allowed.

Return JSON in this exact format:
{{"emotions":[{{"label":"...", "intensity":0.0, "confidence":0.0}}]}}

Input:
{json.dumps(INPUT_JSON)}
""".strip()

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

t0 = time.time()
with torch.no_grad():
    out = model.generate(
        **inputs,
        max_new_tokens=200,
        do_sample=False
    )
lat_ms = (time.time() - t0) * 1000

text = tokenizer.decode(out[0], skip_special_tokens=True)

key = '{"emotions"'
start = text.rfind(key)

if start == -1:
    print("Could not find emotions JSON block.")
    print(text)
else:
    # Brace-matching to find the end of the JSON object starting at `start`
    brace = 0
    end = None
    for i in range(start, len(text)):
        if text[i] == "{":
            brace += 1
        elif text[i] == "}":
            brace -= 1
            if brace == 0:
                end = i + 1
                break

    if end is None:
        print("Found start but could not match JSON braces.")
        print(text[start:start+1000])
    else:
        json_str = text[start:end]
        print("Latency(ms):", round(lat_ms, 1))
        print("JSON_EXTRACTED:")
        print(json_str)

        import json
        try:
            obj = json.loads(json_str)
            print("\nJSON_VALID: True")
            print("Labels returned:", [e.get("label") for e in obj.get("emotions", [])])
        except Exception as e:
            print("\nJSON_VALID: False")
            print("Parse error:", e)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Latency(ms): 17148.7
JSON_EXTRACTED:
{"emotions":[{"label":"stress", "intensity":0.7, "confidence":0.9}, {"label":"frustration", "intensity":0.4, "confidence":0.8}]}

JSON_VALID: True
Labels returned: ['stress', 'frustration']
